In [1]:
import zlib, os

def read_git_object(repo, sha):
    path = os.path.join(repo, '.git', 'objects', sha[:2], sha[2:])
    with open(path, 'rb') as f:
        raw = zlib.decompress(f.read())
    # header is like 'commit 1234\0' or 'tree 1234\0' or 'blob 1234\0'
    null_idx = raw.index(b'\x00')
    header = raw[:null_idx].decode()
    content = raw[null_idx+1:]
    return header, content

REPO = '/workspaces/UI-Desktopwire-frame'
COMMIT_SHA = '88f2c3576ffe90cc5170eb362809bed5e9ba5cf1'

# Step 1: Read commit object
header, content = read_git_object(REPO, COMMIT_SHA)
print('=== COMMIT OBJECT ===')
print('Header:', header)
print('Content:')
print(content.decode())

=== COMMIT OBJECT ===
Header: commit 218
Content:
tree 927b3519b45cd145c81302302c34f8605b92d589
parent 64d4dd565990bf5d5fb6cc47b36cdecf7aed1e98
author Jonggun <0324-0916@lspu.edu.ph> 1771851193 +0000
committer Jonggun <0324-0916@lspu.edu.ph> 1771851193 +0000

1231231



In [2]:
# Step 2: Parse tree SHA from commit and read tree object
lines = content.decode().splitlines()
tree_sha = None
for line in lines:
    if line.startswith('tree '):
        tree_sha = line.split()[1]
        break

print('Tree SHA:', tree_sha)

# Read tree object
tree_header, tree_content = read_git_object(REPO, tree_sha)
print('\n=== TREE OBJECT ===')
print('Header:', tree_header)

# Parse tree entries (binary format: 'mode filename\0' + 20-byte SHA)
import binascii
i = 0
entries = []
while i < len(tree_content):
    null_pos = tree_content.index(b'\x00', i)
    entry_header = tree_content[i:null_pos].decode()
    sha_bytes = tree_content[null_pos+1:null_pos+21]
    sha_hex = binascii.hexlify(sha_bytes).decode()
    mode, name = entry_header.split(' ', 1)
    entries.append((mode, name, sha_hex))
    i = null_pos + 21

print('\nTree entries (mode, name, sha):')
for e in entries:
    print(f'  {e[0]}  {e[1]}  {e[2]}')

Tree SHA: 927b3519b45cd145c81302302c34f8605b92d589

=== TREE OBJECT ===
Header: tree 75

Tree entries (mode, name, sha):
  100644  README.md  5be36f8224fdec23ccd4d24e43c20d4767e6e481
  100644  index.html  9e53b7ad1594f20e39ec0fa973650a3a42b2ce72


In [3]:
# Step 3: Find index.html blob SHA and read its content
blob_sha = None
for mode, name, sha in entries:
    if name == 'index.html':
        blob_sha = sha
        break

print('index.html blob SHA:', blob_sha)

# Check if the blob object exists as a loose object
blob_path = os.path.join(REPO, '.git', 'objects', blob_sha[:2], blob_sha[2:])
if os.path.exists(blob_path):
    blob_header, blob_content = read_git_object(REPO, blob_sha)
    print('\n=== BLOB OBJECT (index.html) ===')
    print('Header:', blob_header)
    print('\nContent:')
    print(blob_content.decode('utf-8', errors='replace'))
else:
    print(f'Blob not found as loose object at {blob_path}')
    print('Checking pack files...')
    pack_dir = os.path.join(REPO, '.git', 'objects', 'pack')
    print('Pack files:', os.listdir(pack_dir))

index.html blob SHA: 9e53b7ad1594f20e39ec0fa973650a3a42b2ce72

=== BLOB OBJECT (index.html) ===
Header: blob 245880

Content:
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8" />
  <meta name="viewport" content="width=device-width, initial-scale=1.0" />
  <title>SUMMIT PT CLINIC – PT Clinic Management System</title>
  <style>
    /* ─── Reset & Base ─────────────────────────────────────────── */
    *, *::before, *::after { box-sizing: border-box; margin: 0; padding: 0; }
    :root {
      --bg:        #f4f5f7;
      --surface:   #ffffff;
      --border:    #d1d5db;
      --border2:   #e5e7eb;
      --text:      #1f2937;
      --muted:     #6b7280;
      --accent:      #1a2d52;
      --accent-dk:  #14203d;
      --danger:     #ef4444;
      --warn:       #f59e0b;
      --ok:         #7a9a28;
      --brand-navy: #1a2d52;
      --brand-green:#7a9a28;
      --brand-gray: #b8c4d0;
      --brand-teal: #c8edf0;
      --sidebar-w: 240px;
      --radius:    8px;
      --shadow:  

In [4]:

# Write recovered blob content to index.html and show preview
BLOB_SHA = '9e53b7ad1594f20e39ec0fa973650a3a42b2ce72'
blob_path = os.path.join(REPO, '.git', 'objects', BLOB_SHA[:2], BLOB_SHA[2:])

if os.path.exists(blob_path):
    _, blob_content = read_git_object(REPO, BLOB_SHA)
    text = blob_content.decode('utf-8', errors='replace')
    
    # Write to index.html
    with open(os.path.join(REPO, 'index.html'), 'w', encoding='utf-8') as f:
        f.write(text)
    print(f'Wrote {len(blob_content)} bytes to index.html')
    print('\n--- FIRST 3000 CHARS ---')
    print(text[:3000])
else:
    print(f'Loose object not found: {blob_path}')
    print('Checking pack files...')
    pack_dir = os.path.join(REPO, '.git', 'objects', 'pack')
    print(os.listdir(pack_dir))


Wrote 245880 bytes to index.html

--- FIRST 3000 CHARS ---
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8" />
  <meta name="viewport" content="width=device-width, initial-scale=1.0" />
  <title>SUMMIT PT CLINIC – PT Clinic Management System</title>
  <style>
    /* ─── Reset & Base ─────────────────────────────────────────── */
    *, *::before, *::after { box-sizing: border-box; margin: 0; padding: 0; }
    :root {
      --bg:        #f4f5f7;
      --surface:   #ffffff;
      --border:    #d1d5db;
      --border2:   #e5e7eb;
      --text:      #1f2937;
      --muted:     #6b7280;
      --accent:      #1a2d52;
      --accent-dk:  #14203d;
      --danger:     #ef4444;
      --warn:       #f59e0b;
      --ok:         #7a9a28;
      --brand-navy: #1a2d52;
      --brand-green:#7a9a28;
      --brand-gray: #b8c4d0;
      --brand-teal: #c8edf0;
      --sidebar-w: 240px;
      --radius:    8px;
      --shadow:    0 1px 4px rgba(0,0,0,.10);
    }
    html, body { height: 100%; f

In [ ]:
# Step 4: If blob is in a pack file, use gitpython or parse pack manually
import struct

def find_in_pack(repo, sha_hex):
    pack_dir = os.path.join(repo, '.git', 'objects', 'pack')
    sha_bin = bytes.fromhex(sha_hex)
    
    for fname in os.listdir(pack_dir):
        if not fname.endswith('.idx'):
            continue
        idx_path = os.path.join(pack_dir, fname)
        pack_path = idx_path[:-4] + '.pack'
        
        with open(idx_path, 'rb') as f:
            idx_data = f.read()
        
        # Check v2 pack index (magic: ff 74 4f 63)
        if idx_data[:4] == b'\xff\x74\x4f\x63':
            # v2 format
            version = struct.unpack('>I', idx_data[4:8])[0]
            # fan-out table: 256 entries of 4 bytes each, starting at offset 8
            total_objects = struct.unpack('>I', idx_data[8 + 255*4: 8 + 256*4])[0]
            # SHA table starts at 8 + 256*4 = 1032
            sha_table_offset = 1032
            
            # Binary search for our SHA
            lo, hi = 0, total_objects
            found_idx = None
            while lo < hi:
                mid = (lo + hi) // 2
                entry_sha = idx_data[sha_table_offset + mid*20 : sha_table_offset + mid*20 + 20]
                if entry_sha == sha_bin:
                    found_idx = mid
                    break
                elif entry_sha < sha_bin:
                    lo = mid + 1
                else:
                    hi = mid
            
            if found_idx is not None:
                # CRC table: after SHA table
                crc_table_offset = sha_table_offset + total_objects * 20
                # Offset table: after CRC table
                offset_table_offset = crc_table_offset + total_objects * 4
                offset = struct.unpack('>I', idx_data[offset_table_offset + found_idx*4 : offset_table_offset + found_idx*4 + 4])[0]
                
                print(f'Found in pack: {fname}, offset: {offset}')
                return pack_path, offset
    
    return None, None

pack_path, offset = find_in_pack(REPO, blob_sha)
print('Pack path:', pack_path, 'Offset:', offset)

In [ ]:
# Read the object from pack file at the given offset
def read_pack_object(pack_path, offset):
    with open(pack_path, 'rb') as f:
        f.seek(offset)
        # Read variable-length size encoding
        byte = ord(f.read(1))
        obj_type = (byte >> 4) & 0x7
        size = byte & 0xf
        shift = 4
        while byte & 0x80:
            byte = ord(f.read(1))
            size |= (byte & 0x7f) << shift
            shift += 7
        
        type_names = {1: 'commit', 2: 'tree', 3: 'blob', 4: 'tag', 6: 'ofs_delta', 7: 'ref_delta'}
        print(f'Object type: {type_names.get(obj_type, obj_type)}, uncompressed size: {size}')
        
        # For ref_delta, read base SHA
        if obj_type == 7:
            base_sha = f.read(20).hex()
            print(f'Base SHA (ref_delta): {base_sha}')
        
        # Read compressed data
        compressed = f.read()
        try:
            data = zlib.decompress(compressed)
        except zlib.error:
            # Try reading just enough
            data = zlib.decompressobj().decompress(compressed)
        
        return obj_type, data

if pack_path:
    obj_type, data = read_pack_object(pack_path, offset)
    print('\n=== BLOB CONTENT (index.html) ===')
    print(data.decode('utf-8', errors='replace'))